In [1]:
import sys
sys.path.insert(0, '../')

In [2]:
import numpy as np
import tensorflow as tf
import datetime
import h5py  
from   tensorflow.keras.layers import Input
from   tensorflow.keras.applications.inception_v3 import InceptionV3
from   utils.data_prep_utils import LABELS, DATES, prep_data
from   utils.train_utils import prep_data_train_val_test, set_curr_time, show_sample
from tqdm import tqdm
np.random.seed(7)

In [3]:
labels = ['zeev', 'or', 'felix', 'ron', 'aviya']

In [4]:
# check seed
np.random.get_state()

('MT19937', array([         7, 4097098180, 3572822661, 1142383841, 2757143652,
        4148075075, 1140792134,  204456458, 3824478202, 1670563398,
         888486157,  732067628, 4212363368, 2649636164,   45261484,
        2889628139,  270502397, 3207910882,  258818418, 1883165709,
        3298301136, 2297045588, 2300154372, 1281741429, 1391991772,
        2178368330, 2749830018, 3692704155, 1443358996, 1752052326,
          21234113, 1561066052,  311143001, 1135793726, 3656334845,
        3716309849, 3675413670, 1716902462, 3714306561, 1506995441,
         194243288, 1173438305, 4005060874, 1307253176, 1318332201,
         998234869, 1045528535, 1718779650, 1849276255,  192456263,
        2542180661,  882500070, 1340356082, 2743948308, 3176919780,
        2525883125, 4185711275, 1445549185, 1812387514,  179948290,
         234826246, 3243428507, 1044203062,  205101453, 1421937121,
        2207368609,  381432977, 1280923256, 1473568513, 2874342213,
        3542557257, 1741693305, 37800

In [11]:
# prep_data(DATES, labels, frames_size=128)

In [5]:
x_train, x_valid, x_test, y_train, y_valid, y_test = prep_data_train_val_test(DATES, labels, frames_size=128)

100%|██████████| 3000/3000 [00:01<00:00, 1527.82it/s]
0it [00:00, ?it/s]
100%|██████████| 3000/3000 [00:01<00:00, 1738.19it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 30000/30000 [00:18<00:00, 1616.71it/s]
0it [00:00, ?it/s]
100%|██████████| 21000/21000 [00:14<00:00, 1468.37it/s]
0it [00:00, ?it/s]
100%|██████████| 21000/21000 [00:13<00:00, 1538.41it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [86]:
###### check that train - valid- test are not mixed

# check that train -> valid- test are not mixed
train_check = np.random.randint(low=0, high=x_train.shape[0], size=(int(x_train.shape[0]*0.1),))
for i in tqdm(train_check):
    for j in range(x_valid.shape[0]):
        if np.equal(x_train[i], x_valid[j]).all() :
            print("train -> valid")

    for k in range(x_test.shape[0]):
        if np.equal(x_train[i],x_test[k]).all() :
            print("train -> test")
        
# check that  valid -> train- test are not mixed
valid_check = np.random.randint(low=0, high=x_valid.shape[0], size=(int(x_valid.shape[0]*0.1),))
for i in tqdm(valid_check):
    for j in range(x_train.shape[0]):
        if np.equal(x_valid[i], x_train[j]).all() :
            print(" valid ->  train")


    for k in range(x_test.shape[0]):
        if np.equal(x_valid[i], x_test[k]).all() :
            print(" valid ->  test")
        
        
# check that  test -> train-valid test are not mixed
test_check = np.random.randint(low=0, high=x_test.shape[0], size=(int(x_test.shape[0]*0.1),))
for i in tqdm(test_check):
    for j in range(x_train.shape[0]):
        if np.equal(x_test[i], x_train[j]).all() :
            print("test -> train")


    for k in range(x_valid.shape[0]):
        if np.equal(x_test[i], x_valid[k]).all() :
            print("test -> valid")
        




  0%|          | 0/11280 [00:00<?, ?it/s]

  0%|          | 1/11280 [00:00<28:08,  6.68it/s]

  0%|          | 2/11280 [00:00<28:11,  6.67it/s]

  0%|          | 3/11280 [00:00<28:10,  6.67it/s]

  0%|          | 4/11280 [00:00<28:11,  6.67it/s]

  0%|          | 5/11280 [00:00<28:11,  6.67it/s]

  0%|          | 6/11280 [00:00<28:15,  6.65it/s]

  0%|          | 7/11280 [00:01<28:11,  6.66it/s]

  0%|          | 8/11280 [00:01<28:11,  6.66it/s]

  0%|          | 9/11280 [00:01<28:10,  6.67it/s]

  0%|          | 10/11280 [00:01<28:10,  6.67it/s]

  0%|          | 11/11280 [00:01<28:10,  6.67it/s]

  0%|          | 12/11280 [00:01<28:13,  6.65it/s]

  0%|          | 13/11280 [00:01<28:10,  6.66it/s]

  0%|          | 14/11280 [00:02<28:11,  6.66it/s]

  0%|          | 15/11280 [00:02<28:13,  6.65it/s]

  0%|          | 16/11280 [00:02<28:13,  6.65it/s]

  0%|          | 17/11280 [00:02<28:12,  6.65it/s]

  0%|          | 18/11280 [00:02<28:14,  6.65it/s]

  0%|          | 19/11280 [0

  1%|          | 140/11280 [00:21<27:54,  6.65it/s]

  1%|▏         | 141/11280 [00:21<28:01,  6.63it/s]

  1%|▏         | 142/11280 [00:21<27:55,  6.65it/s]

  1%|▏         | 143/11280 [00:21<27:53,  6.65it/s]

  1%|▏         | 144/11280 [00:21<27:51,  6.66it/s]

  1%|▏         | 145/11280 [00:21<27:49,  6.67it/s]

  1%|▏         | 146/11280 [00:21<27:51,  6.66it/s]

  1%|▏         | 147/11280 [00:22<27:48,  6.67it/s]

  1%|▏         | 148/11280 [00:22<27:49,  6.67it/s]

  1%|▏         | 149/11280 [00:22<27:48,  6.67it/s]

  1%|▏         | 150/11280 [00:22<27:48,  6.67it/s]

  1%|▏         | 151/11280 [00:22<27:49,  6.67it/s]

  1%|▏         | 152/11280 [00:22<27:50,  6.66it/s]

  1%|▏         | 153/11280 [00:22<27:51,  6.66it/s]

  1%|▏         | 154/11280 [00:23<27:52,  6.65it/s]

  1%|▏         | 155/11280 [00:23<27:53,  6.65it/s]

  1%|▏         | 156/11280 [00:23<27:50,  6.66it/s]

  1%|▏         | 157/11280 [00:23<27:54,  6.64it/s]

  1%|▏         | 158/11280 [00:23<27:49,  6.66

  2%|▏         | 278/11280 [00:41<27:45,  6.61it/s]

  2%|▏         | 279/11280 [00:41<27:41,  6.62it/s]

  2%|▏         | 280/11280 [00:42<27:37,  6.64it/s]

  2%|▏         | 281/11280 [00:42<27:45,  6.60it/s]

  2%|▎         | 282/11280 [00:42<27:43,  6.61it/s]

  3%|▎         | 283/11280 [00:42<27:46,  6.60it/s]

  3%|▎         | 284/11280 [00:42<27:42,  6.61it/s]

  3%|▎         | 285/11280 [00:42<27:39,  6.62it/s]

  3%|▎         | 286/11280 [00:42<27:35,  6.64it/s]

  3%|▎         | 287/11280 [00:43<27:43,  6.61it/s]

  3%|▎         | 288/11280 [00:43<27:44,  6.60it/s]

  3%|▎         | 289/11280 [00:43<27:45,  6.60it/s]

  3%|▎         | 290/11280 [00:43<27:47,  6.59it/s]

  3%|▎         | 291/11280 [00:43<27:48,  6.59it/s]

  3%|▎         | 292/11280 [00:43<27:46,  6.60it/s]

  3%|▎         | 293/11280 [00:44<27:45,  6.59it/s]

  3%|▎         | 294/11280 [00:44<27:46,  6.59it/s]

  3%|▎         | 295/11280 [00:44<27:51,  6.57it/s]

  3%|▎         | 296/11280 [00:44<27:48,  6.58

  4%|▎         | 416/11280 [01:02<27:28,  6.59it/s]

  4%|▎         | 417/11280 [01:02<27:32,  6.57it/s]

  4%|▎         | 418/11280 [01:02<27:30,  6.58it/s]

  4%|▎         | 419/11280 [01:03<27:31,  6.58it/s]

  4%|▎         | 420/11280 [01:03<27:32,  6.57it/s]

  4%|▎         | 421/11280 [01:03<27:32,  6.57it/s]

  4%|▎         | 422/11280 [01:03<27:31,  6.57it/s]

  4%|▍         | 423/11280 [01:03<27:31,  6.57it/s]

  4%|▍         | 424/11280 [01:03<27:25,  6.60it/s]

  4%|▍         | 425/11280 [01:04<27:24,  6.60it/s]

  4%|▍         | 426/11280 [01:04<27:19,  6.62it/s]

  4%|▍         | 427/11280 [01:04<27:14,  6.64it/s]

  4%|▍         | 428/11280 [01:04<27:12,  6.65it/s]

  4%|▍         | 429/11280 [01:04<27:11,  6.65it/s]

  4%|▍         | 430/11280 [01:04<27:13,  6.64it/s]

  4%|▍         | 431/11280 [01:04<27:11,  6.65it/s]

  4%|▍         | 432/11280 [01:05<27:10,  6.65it/s]

  4%|▍         | 433/11280 [01:05<27:19,  6.62it/s]

  4%|▍         | 434/11280 [01:05<27:14,  6.64

  5%|▍         | 554/11280 [01:23<26:51,  6.66it/s]

  5%|▍         | 555/11280 [01:23<26:51,  6.65it/s]

  5%|▍         | 556/11280 [01:23<26:49,  6.66it/s]

  5%|▍         | 557/11280 [01:23<26:57,  6.63it/s]

  5%|▍         | 558/11280 [01:24<27:00,  6.62it/s]

  5%|▍         | 559/11280 [01:24<27:01,  6.61it/s]

  5%|▍         | 560/11280 [01:24<27:01,  6.61it/s]

  5%|▍         | 561/11280 [01:24<27:03,  6.60it/s]

  5%|▍         | 562/11280 [01:24<27:05,  6.59it/s]

  5%|▍         | 563/11280 [01:24<27:05,  6.59it/s]

  5%|▌         | 564/11280 [01:24<27:01,  6.61it/s]

  5%|▌         | 565/11280 [01:25<27:02,  6.60it/s]

  5%|▌         | 566/11280 [01:25<26:55,  6.63it/s]

  5%|▌         | 567/11280 [01:25<26:52,  6.64it/s]

  5%|▌         | 568/11280 [01:25<26:52,  6.64it/s]

  5%|▌         | 569/11280 [01:25<26:59,  6.62it/s]

  5%|▌         | 570/11280 [01:25<26:53,  6.64it/s]

  5%|▌         | 571/11280 [01:26<26:56,  6.63it/s]

  5%|▌         | 572/11280 [01:26<26:52,  6.64

...

 60%|█████▉    | 845/1410 [09:27<06:20,  1.48it/s]

 60%|██████    | 846/1410 [09:28<06:19,  1.49it/s]

 60%|██████    | 847/1410 [09:29<06:18,  1.49it/s]

 60%|██████    | 848/1410 [09:29<06:17,  1.49it/s]

 60%|██████    | 849/1410 [09:30<06:17,  1.49it/s]

 60%|██████    | 850/1410 [09:31<06:16,  1.49it/s]

 60%|██████    | 851/1410 [09:31<06:16,  1.49it/s]

 60%|██████    | 852/1410 [09:32<06:16,  1.48it/s]

 60%|██████    | 853/1410 [09:33<06:15,  1.49it/s]

 61%|██████    | 854/1410 [09:33<06:14,  1.49it/s]

 61%|██████    | 855/1410 [09:34<06:13,  1.49it/s]

 61%|██████    | 856/1410 [09:35<06:12,  1.49it/s]

 61%|██████    | 857/1410 [09:35<06:12,  1.49it/s]

 61%|██████    | 858/1410 [09:36<06:12,  1.48it/s]

 61%|██████    | 859/1410 [09:37<06:11,  1.48it/s]

 61%|██████    | 860/1410 [09:37<06:11,  1.48it/s]

 61%|██████    | 861/1410 [09:38<06:10,  1.48it/s]

 61%|██████    | 862/1410 [09:39<06:09,  1.48it/s]

 61%|██████    | 863/1410 [09:39<06:08,  1.49it/s]

 61%|██████▏

 70%|██████▉   | 986/1410 [11:02<04:46,  1.48it/s]

 70%|███████   | 987/1410 [11:03<04:45,  1.48it/s]

 70%|███████   | 988/1410 [11:04<04:45,  1.48it/s]

 70%|███████   | 989/1410 [11:04<04:44,  1.48it/s]

 70%|███████   | 990/1410 [11:05<04:43,  1.48it/s]

 70%|███████   | 991/1410 [11:06<04:42,  1.48it/s]

 70%|███████   | 992/1410 [11:06<04:42,  1.48it/s]

 70%|███████   | 993/1410 [11:07<04:43,  1.47it/s]

 70%|███████   | 994/1410 [11:08<04:41,  1.48it/s]

 71%|███████   | 995/1410 [11:08<04:41,  1.47it/s]

 71%|███████   | 996/1410 [11:09<04:40,  1.48it/s]

 71%|███████   | 997/1410 [11:10<04:39,  1.48it/s]

 71%|███████   | 998/1410 [11:10<04:37,  1.48it/s]

 71%|███████   | 999/1410 [11:11<04:37,  1.48it/s]

 71%|███████   | 1000/1410 [11:12<04:36,  1.49it/s]

 71%|███████   | 1001/1410 [11:12<04:34,  1.49it/s]

 71%|███████   | 1002/1410 [11:13<04:34,  1.49it/s]

 71%|███████   | 1003/1410 [11:14<04:33,  1.49it/s]

 71%|███████   | 1004/1410 [11:14<04:32,  1.49it/s]

 71%|██

 80%|███████▉  | 1125/1410 [12:37<03:34,  1.33it/s]

 80%|███████▉  | 1126/1410 [12:38<03:34,  1.32it/s]

 80%|███████▉  | 1127/1410 [12:39<03:52,  1.22it/s]

 80%|████████  | 1128/1410 [12:39<03:46,  1.25it/s]

 80%|████████  | 1129/1410 [12:40<03:37,  1.29it/s]

 80%|████████  | 1130/1410 [12:41<03:39,  1.28it/s]

 80%|████████  | 1131/1410 [12:42<03:29,  1.33it/s]

 80%|████████  | 1132/1410 [12:42<03:23,  1.37it/s]

 80%|████████  | 1133/1410 [12:43<03:19,  1.39it/s]

 80%|████████  | 1134/1410 [12:44<03:14,  1.42it/s]

 80%|████████  | 1135/1410 [12:44<03:12,  1.43it/s]

 81%|████████  | 1136/1410 [12:45<03:10,  1.44it/s]

 81%|████████  | 1137/1410 [12:46<03:08,  1.45it/s]

 81%|████████  | 1138/1410 [12:46<03:07,  1.45it/s]

 81%|████████  | 1139/1410 [12:47<03:06,  1.45it/s]

 81%|████████  | 1140/1410 [12:48<03:05,  1.46it/s]

 81%|████████  | 1141/1410 [12:48<03:03,  1.46it/s]

 81%|████████  | 1142/1410 [12:49<03:02,  1.47it/s]

 81%|████████  | 1143/1410 [12:50<03:02,  1.46

 90%|████████▉ | 1263/1410 [14:16<01:51,  1.32it/s]

 90%|████████▉ | 1264/1410 [14:17<01:51,  1.31it/s]

 90%|████████▉ | 1265/1410 [14:18<01:50,  1.31it/s]

 90%|████████▉ | 1266/1410 [14:19<01:49,  1.31it/s]

 90%|████████▉ | 1267/1410 [14:19<01:48,  1.31it/s]

 90%|████████▉ | 1268/1410 [14:20<01:48,  1.31it/s]

 90%|█████████ | 1269/1410 [14:21<01:46,  1.32it/s]

 90%|█████████ | 1270/1410 [14:22<01:46,  1.31it/s]

 90%|█████████ | 1271/1410 [14:22<01:44,  1.32it/s]

 90%|█████████ | 1272/1410 [14:23<01:41,  1.36it/s]

 90%|█████████ | 1273/1410 [14:24<01:40,  1.37it/s]

 90%|█████████ | 1274/1410 [14:24<01:40,  1.36it/s]

 90%|█████████ | 1275/1410 [14:25<01:41,  1.33it/s]

 90%|█████████ | 1276/1410 [14:26<01:40,  1.33it/s]

 91%|█████████ | 1277/1410 [14:27<01:41,  1.31it/s]

 91%|█████████ | 1278/1410 [14:28<01:39,  1.33it/s]

 91%|█████████ | 1279/1410 [14:28<01:39,  1.32it/s]

 91%|█████████ | 1280/1410 [14:29<01:37,  1.33it/s]

 91%|█████████ | 1281/1410 [14:30<01:37,  1.33

 99%|█████████▉| 1401/1410 [15:57<00:06,  1.47it/s]

 99%|█████████▉| 1402/1410 [15:57<00:05,  1.48it/s]

100%|█████████▉| 1403/1410 [15:58<00:04,  1.48it/s]

100%|█████████▉| 1404/1410 [15:59<00:04,  1.48it/s]

100%|█████████▉| 1405/1410 [15:59<00:03,  1.47it/s]

100%|█████████▉| 1406/1410 [16:00<00:02,  1.47it/s]

100%|█████████▉| 1407/1410 [16:01<00:02,  1.47it/s]

100%|█████████▉| 1408/1410 [16:01<00:01,  1.47it/s]

100%|█████████▉| 1409/1410 [16:02<00:00,  1.48it/s]

100%|██████████| 1410/1410 [16:03<00:00,  1.48it/s]

In [6]:
x_train_rgb = np.repeat(x_train[..., np.newaxis], 3, -1)
x_val_rgb = np.repeat(x_valid[..., np.newaxis], 3, -1)
x_test_rgb = np.repeat(x_test[..., np.newaxis], 3, -1)

In [7]:
input_tensor = Input(shape=(128, 128, 3))
inceptionv3_model = InceptionV3(input_tensor=input_tensor, weights=None, include_top=True, classes=len(labels))

In [8]:
for layer in inceptionv3_model.layers:
    layer.trainable = True

In [9]:
inceptionv3_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [10]:
def set_curr_time():
    dt = datetime.datetime.now()
    curr_dt = '{0}{1}{2}_{3}_{4}'.format(datetime.datetime.now().year, datetime.datetime.now().month,
                                       datetime.datetime.now().day, datetime.datetime.now().hour,
                                      datetime.datetime.now().minute)
    return curr_dt

In [11]:
curr_dt = set_curr_time()
tb_callback = tf.keras.callbacks.TensorBoard(log_dir='./logs/{0}_{1}'.format('inceptionv3', curr_dt),
                                             histogram_freq=0, write_graph=True)

inceptionv3_model.fit(x_train_rgb, y_train, validation_data=(x_val_rgb, y_valid), batch_size=128, epochs=70,
          callbacks=[tb_callback])

In [ ]:
print(inceptionv3_model.evaluate(x_test_rgb, y_test))

In [ ]:

inceptionv3_model.save('../models/inceptionv3_{0}.h5'.format(set_curr_time()))